In [8]:
import pandas as pd
import numpy as np

In [9]:
pip install pycham

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.6 MB/s eta 0:00:00


In [19]:
# class Variable:
#   def __init__(self, data):
#     if data is not None:
#       if not isinstance(data, np.ndarray):
#         raise TypeError('{} in not supported'.format(type(data)))

#     self.data = data
#     self.grad = None
#     self.creator = None

class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

      funcs = [self.creator]
      while funcs:
        f = funcs.pop()
        x, y = f.input, f.output
        x.grad = f.backward(y.grad)

        if x.creator is not None:
          funcs.append(x.creator)


def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x

# Function 클래스 수정 (인수와 반환값을 리스트로 변경)
- Variable 에서 실제 데이터를 추출
- 순전파 메서드에서 구체적인 계산을 수행
- 계산 결과를 Variable 넣음

In [20]:
class Function:
  def __call__(self, inputs):
    xs = [ x.data for x in inputs ]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
      self.inputs = inputs
      self.outputs = outputs
      return outputs

    def forward(self, x):
      raise NotImplementedError()

    def backward(self, gy):
      raise NotImplementedError()


# Add 클래스 구현
- 인수, 반환값 = List or Tupple
- 인수는 변수가 두개 담긴 list
- 결과 = tupple

In [30]:
class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return (y, )


- 개선점 필요

입력시 변수를 리스트로 전달하도록 요청

반환값 튜플로 전달

사용시 복잡


In [22]:

xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


# Add class 개선하기

## 첫번째 : 함수 사용 쉽게

개선사항
- 리스트나 튜플을 거치지 않고 인수와 결과를 직접 주고받도록 함.

In [29]:
class Function:
  def __call__(self ,* inputs): # 함수 정의할 때 인수 앞 별표(*) 붙이기
    xs = [ x.data for x in inputs ]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
      self.inputs = inputs
      self.outputs = outputs
      return outputs if len(outputs) > 1 else outputs[0] # 반환값이 하나가 아니면 변수를 직접 돌려줌

    def forward(self, x):
      raise NotImplementedError()

    def backward(self, gy):
      raise NotImplementedError()


class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return (y, )


전에 비해서 확실히 계산이 편해짐

In [31]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0,x1)
print(y.data)

5


## 두 번째 : 함수를 구현하기 쉽도록

- Add 클래스 구현을 위한 개선
forward 메서드의 코드를 입력도 변수로 받고, 결과도 변수로 반환

- Funtion 클래스 수정

리스트 언팩사용

forward 반환값이 튜플이 아닌 경우 튜플로 변경

In [34]:
from ast import YieldFrom
class Function:
  def __call__(self ,* inputs): # 함수 정의할 때 인수 앞 별표(*) 붙이기
    xs = [ x.data for x in inputs ]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):  # 함수 호출할 때 별표(*)
      ys = (ys,)                   # 튜플 값이 아닌 경우 튜플로 변경
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
      self.inputs = inputs
      self.outputs = outputs
      return outputs if len(outputs) > 1 else outputs[0] # 반환값이 하나가 아니면 변수를 직접 돌려줌

    def forward(self, x):
      raise NotImplementedError()

    def backward(self, gy):
      raise NotImplementedError()


class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y


Add -> add 함수로 개선 완료

In [37]:
def add(x0, x1):
  return Add()(x0,x1)

x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0,x1)
print(y.data)

5


# 가변 길이 인수 (역전파 편) / (Variable)

- 가변 길이 인수에 대응한 Add 클래스의 역전파

<덧셈 계산 그래프에서 순전파와 역전파>

<다변수 함수>

여러개의 변수에 대응할 수 있도록 수정

In [41]:
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

      funcs = [self.creator]
      while funcs:
        f = funcs.pop()
        gys = [output.grad for output in f.outputs] # 여러 개의 변수에 대응 가능
        gxs = f.backward(*gys)
        if not isinstance(gxs, tuple):
          gxs = (gxs, )

        for x, gx in zip(f.inputs, gxs):
          x.grad = gx

        if x.creator is not None:
          funcs.append(x.creator)

class Square(Function):
  def forward(self,x):
    return x **2

  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2 * x * gy
    return gx

In [ ]:
def add(x0, x1):
  return Add()(x0,x1)
def square(x):
  return Square()(x)

# z^2 = x^2 + y^2
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))
z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

# 같은 변수 반복 사용 문제

<문제원인>

backward에서 출력 쪽에서 전해지는 미분값을 그대로 대입함.

같은 변수 반복하여 사용시 전파되는 미분값이 덮어 써짐

<해결책>

미분값을 처음 설정하는 경우는 출력에서 전해지는 미분값을 그대로 대입

처음 이후 부터는 전달된 미분값을 더해주도록 수정

In [44]:
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

      funcs = [self.creator]
      while funcs:
        f = funcs.pop()
        gys = [output.grad for output in f.outputs] # 여러 개의 변수에 대응 가능
        gxs = f.backward(*gys)
        if not isinstance(gxs, tuple):
          gxs = (gxs, )

        for x, gx in zip(f.inputs, gxs):
          if x.grad is None: # 미분값(grad)을 처음 설정하는 경우 출력에서 전해지는 미분값으로 대입
            x.grad = gx      # 처음 이후 부터는 전달된 미분값을 더해주도록 수정
          else:
            x.grad = x.grad + gx

        if x.creator is not None:
          funcs.append(x.creator)

class Square(Function):
  def forward(self,x):
    return x **2

  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2 * x * gy
    return gx

In [ ]:
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
prind(x.grad)

미분값 재설정

<역전파 시 미분값을 더해주는 코드 문제점>

같은 변수를 사용하여 다른 계산을 할 경우 계산이 꼬이는 문제 발생

< 해결 방법 >

Variable 클래스에 미분값을 초기화 하는 cleargrad 메서드 추가

- x.cleargrad()

# 복잡한 계산 그래프(이론)

지금까지는 일직선 계싼 그래프의 미분 계산

복잡하게 연결된 계산 그래프의 미분 계산이 필요

<역전파의 올바른 순서>

- 복잡하게 연결된 그래프의 올바른 순서

Variable 클래스의 backward 메서드 구현

- Backward 메서드에서 중첩 메서드 add_func 함수 추가

- add_func 함수가 함수 리스트를 세대 순으로 정렬하는 역할

- 정렬이 되어서 func.pop() 을 수행시 세대가 가장 큰 함수를 꺼냄

In [49]:
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{} in not supported'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0  # generation 을 0으로 초기화

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # set_creator 호출될때 부모 함수의 세대보다 1만큼 큰 값을 설정

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

      funcs = []
      seen_set = set()

      def add_func(f):
        if f not in seen_set:
          funcs.append(f)
          seen_set.add(f)
          funcs.sort(key=lambda x:x.generation)

      while funcs:
        f = funcs.pop()
        gys = [output.grad for output in f.outputs] # 여러 개의 변수에 대응 가능
        gxs = f.backward(*gys)
        if not isinstance(gxs, tuple):
          gxs = (gxs, )

        for x, gx in zip(f.inputs, gxs):
          if x.grad is None: # 미분값(grad)을 처음 설정하는 경우 출력에서 전해지는 미분값으로 대입
            x.grad = gx      # 처음 이후 부터는 전달된 미분값을 더해주도록 수정
          else:
            x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator)

class Square(Function):
  def forward(self,x):
    return x **2

  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2 * x * gy
    return gx

입력 변수가 둘이상이라면 가장 큰 generation 수를 선택

In [48]:
class Function:
  def __call__(self, inputs):
    xs = [ x.data for x in inputs ]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    self.generation = max([x.generation for x in inputs]) #
    for output in outputs:
      output.set_creator(self)
      self.inputs = inputs
      self.outputs = outputs
      return outputs

    def forward(self, x):
      raise NotImplementedError()

    def backward(self, gy):
      raise NotImplementedError()

